In [1]:
import json
import re
import string
import numpy as np
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory # Rumus Library
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string # Rumus Library
import sys # Rumus supaya baca command promp


stemmer = StemmerFactory().create_stemmer() # Object stemmer
remover = StopWordRemoverFactory().create_stop_word_remover() # objek stopword

In [50]:
data = pd.read_excel('bahanbaku.xlsx');
list_dokumen = data.Bahan.tolist()
list_dokumen = [tem.replace(u"\xa0",u"") for tem in list_dokumen]
list_dokumen = [tem.replace(u"\n",u", ") for tem in list_dokumen]
list_dokumen = [str(tem) for tem in list_dokumen]
list_dokumen[0]

'30 gram Bayam, 25 gram Butter Tawar, 30 gram Daun Bawang, 25 gram Kacang Polong Beku, 200 ml Kaldu, 25 gram Ubi'

In [35]:
def preprocess(doc): #fungsi untuk preprocesing dokumen
    listStem = [] # variabel untuk menampung data preprocesing
    for i in doc:
        lowerku = i.lower() # membuat huruf kecil
        s = re.sub(r'[^\w\s]','',lowerku) # normalisasi text dari code
        textStemmed = stemmer.stem(s) # steming kata
        textClean = remover.remove(textStemmed) # membuang kata tdak penting
        if textClean != "-": 
            listStem.append(textClean) #menambah data kata yang sudah di steming di dalam variabel liststem
    return listStem # mengembalikan nilai listStem

def replaceMultiple(mainString, toBeReplaces, newString):
    # Iterate over the strings to be replaced
    for elem in toBeReplaces :
        # Check if string is in the main string
        if elem in mainString :
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return  mainString

In [36]:
pre_bahan = preprocess(list_dokumen)
pre_bahan

['30 gram bayam 25 gram butter tawar 30 gram daun bawang 25 gram kacang polong beku 200 ml kaldu 25 gram ubi',
 '12 sendok teh garam halus 500 cc kaldu 10 gram margarin 50 gram tahu 20 gram tepung beras 50 gram tomat 40 gram wortel',
 '12 buah jeruk baby 1 buah pisang ambon',
 '1 butir kuning telur 50 gram gula kastor 75 gram keju cheddar 130 gram mentega tawar 25 gram tepung jagung 150 gram tepung terigu',
 '50 ml air 14 sendok teh kayumanis 2 buah pisang ambon',
 '100 cc air masak hangat 50 gram aprikot 100 cc asi perah 50 gram pepaya 3 sendok takar peres susu formula 40 gram promina beras merah',
 '115 gram brokoli 175 gram kentang 300 cc susu formula cair 115 gram wortel',
 '100 ml air 50 gram havermut instan 50 gram pepaya 150 ml susu cair',
 '50 gram bayam 50 gram ikan dor 100 ml kaldu ikan 50 gram keju parut 1 buah kentang 1 sendok teh tepung maizena',
 '1 buah apel 250 ml kaldu ayam 30 gram kentang 30 gram wortel',
 '200 cc air matang hangat 1 buah pisang ambon 2 lembar roti ta

In [37]:
list_bahan = []
for sampel in pre_bahan:
    result = ''.join([i for i in sampel if not i.isdigit()])
    otherStr = replaceMultiple(result, ['gram','ml','cc','buah','sendok teh','sendok makan','sendok takar','butir','cangkir','siung','batang'] ,"")
    a = re.sub(' +',' ',otherStr) #untuk menghilangkan double spasi dikalimat
    b = a.lstrip() #untuk hilangkan spasi di depan kalimat
    list_bahan.append(b)
print(list_bahan)

['bayam butter tawar daun bawang kacang polong beku kaldu ubi', 'garam halus kaldu margarin tahu tepung beras tomat wortel', 'jeruk baby pisang ambon', 'kuning telur gula kastor keju cheddar mentega tawar tepung jagung tepung terigu', 'air kayumanis pisang ambon', 'air masak hangat aprikot asi perah pepaya peres susu formula promina beras merah', 'brokoli kentang susu formula cair wortel', 'air havermut instan pepaya susu cair', 'bayam ikan dor kaldu ikan keju parut kentang tepung maizena', 'apel kaldu ayam kentang wortel', 'air matang hangat pisang ambon lembar roti tawar susu formula', 'asi stroberi', 'asi air matang gula pasir halus jagung manis', 'asi air wortel', 'alpukat matang susu formula tepung beras merah', 'ikan salmon kaldu ikan kentang', 'air fillet kakap potong kecilkecil garam kacang polong labu kuning potong dadu wortel', 'air jeruk manis pisang ambon', 'air panas bayam merah ikan kakap labu kuning margarin', 'air melon susu bubuk formula tepung beras', 'asi daging tuna

In [38]:
import math

def dice_distance(a, b):
    c = a.intersection(b)
    return float(2 * len(c)) / (len(a) + len(b))

def search(query, datanya):
    lol = preprocess([query])
    alldata = []
    alldata.append(set(lol[0].split(" ")))
    for i in datanya:
        lol2 = preprocess([i])
        alldata.append(lol2[0].split(" "))
    
    score = []
#     print alldata
    for tf in range(1,len(alldata)):
        score.append(dice_distance(alldata[0],alldata[tf]))
#     print len(alldata)
#         print score
    return score

In [39]:
semuanya = []
n = search("daging ayam makaroni tahu", list_bahan)
frame = pd.DataFrame(data['Bahan'])
# print data['Temuan']
frame['Score'] = n
frame

,Bahan,Score
0,30 gram Bayam\n25 gram Butter Tawar\n30 gram D...,0.000000
1,1/2 sendok teh Garam Halus\n500 cc Kaldu\n10 g...,0.153846
2,1/2 buah Jeruk Baby\n1 buah Pisang Ambon,0.000000
3,1 butir kuning telur\n50 gram Gula Kastor\n75 ...,0.000000
4,50 ml Air\n1/4 sendok teh Kayumanis\n2 buah Pi...,0.000000
5,100 cc Air Masak Hangat\n50 gram Aprikot\n100 ...,0.000000
6,115 gram Brokoli\n175 gram Kentang\n300 cc Sus...,0.000000
7,100 ml Air\n50 gram Havermut Instan\n50 gram P...,0.000000
8,50 gram Bayam\n50 gram Ikan Dori\n100 ml Kaldu...,0.000000
9,1 buah Apel\n250 ml Kaldu Ayam\n30 gram Kentan...,0.222222


In [40]:
i = 0
alldrop = []
for testscore in frame['Score']:
            if testscore<=0.5:
                alldrop.append(i)
#                 frame.drop(frame.index[i])
            i = i + 1

In [41]:
newframe = frame.drop(frame.index[alldrop])
rank = newframe.sort_values(by=['Score'], ascending=[False])
newframe
rank

,Bahan,Score
88,\n350 cc Air\n50 gram Daging Ayam\n50 gram Nas...,0.545455
109,25 gram Daging Ayam\n25 gram Kacang Polong\n30...,0.533333


In [42]:
ket = []
for data in frame['Score']:
    if data >= 0.5:
        z = "Relevan"
    else:
        z = "Tidak Relevan"
    ket.append(z) # Harus di simpen dulu di list biar jumlahnya sama seperti data frame
frame['Keterangan'] = ket # baru di assignment ket ke dalam frame['keterangan']

In [45]:
rank = frame.sort_values(by=['Score'], ascending=[False])
rank

,Bahan,Score,Keterangan
88,\n350 cc Air\n50 gram Daging Ayam\n50 gram Nas...,0.545455,Relevan
109,25 gram Daging Ayam\n25 gram Kacang Polong\n30...,0.533333,Relevan
61,120 ml Air\n25 gram Brokoli\n40 gram Daging Ay...,0.500000,Relevan
111,2 sendok makan Beras\n25 gram Daging Ayam\n300...,0.400000,Tidak Relevan
52,250 ml Air\n25 gram Buncis Potong Kecil\n30 gr...,0.352941,Tidak Relevan
20,60 ml ASI\n40 gram Daging Tuna\n1/2 sendok mak...,0.333333,Tidak Relevan
89,250 cc Air\n35 gram Brokoli\n35 gram Daging Ay...,0.333333,Tidak Relevan
45,\n2 sendok makan Beras Putih\n1 sendok makan D...,0.307692,Tidak Relevan
37,50 gram Bayam\n100 gram Daging Sapi\n100 gram ...,0.307692,Tidak Relevan
64,10 gram Bayam\n10 gram Daging Ayam\n125 ml Kal...,0.307692,Tidak Relevan
